In [1]:
import numpy as np
import pandas as pd
import os, gc
from glob import glob
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
import sys
# sys.path.append(f'/home/{os.environ.get("USER")}/PythonLibrary')
# import lgbextension as ex
import lightgbm as lgb
from multiprocessing import cpu_count
from time import sleep
#sleep(60 * 5)
import time
from sklearn.metrics import roc_auc_score
SEED = np.random.randint(9999)
print('SEED:', SEED)
import utils

NFOLD = 5

LOOP = 1

SEED: 6085


In [3]:


# =============================================================================
# load
# =============================================================================

# y_train = pd.concat([pd.read_csv("Y_train.csv"),pd.read_csv("Y_val.csv")]).reset_index(drop = True)
# X_train = pd.concat([pd.read_csv("X_train.csv"),pd.read_csv("X_val.csv")]).reset_index(drop = True)

# var_names = X_train.columns.values

df = pd.read_csv("C:\\Users\\nigel\\Desktop\\中兴\\Data\\train.csv")

X_train = df.drop(columns = ['label']).reset_index(drop = True)

y_train = df['label']

var_names = X_train.columns.values

In [4]:
# ModelExtractionCallback class definition should be here.
class ModelExtractionCallback(object):
    """
    original author : momijiame
    ref : https://blog.amedama.jp/entry/lightgbm-cv-model
    description : Class for callback to extract trained models from lightgbm.cv(). 
    note: This class depends on private class '_CVBooster', so there are some future risks. 
    """

    def __init__(self):
        self._model = None

    def __call__(self, env):
        # _CVBooster の参照を保持する
        self._model = env.model

    def _assert_called_cb(self):
        if self._model is None:
            # コールバックが呼ばれていないときは例外にする
            raise RuntimeError('callback has not called yet')

    @property
    def boosters_proxy(self):
        self._assert_called_cb()
        # Booster へのプロキシオブジェクトを返す
        return self._model

    @property
    def raw_boosters(self):
        self._assert_called_cb()
        # Booster のリストを返す
        return self._model.boosters

    @property
    def best_iteration(self):
        self._assert_called_cb()
        # Early stop したときの boosting round を返す
        return self._model.best_iteration



得到一个gain和lost_list:\
gain:表示特征的gain，越大越好\
lost_list: auc值

In [10]:
param = {
    'bagging_freq': 5,
    'bagging_fraction': 0.9,
    'boost_from_average':'false',
    'boost': 'gbdt',
    'feature_fraction': 1.0,
    'learning_rate': 0.05,
    'max_depth': -1,
    'metric':'auc_mu',
    'min_data_in_leaf': 10,
    'min_sum_hessian_in_leaf': 10.0,
    'num_leaves': 4,
    'n_jobs': 12,
    'tree_learner': 'serial',
    'objective': 'multiclass',
    'return_cvbooster':True,
    'num_classes':6,
    'verbosity': -1}


NROUND = 99999
ESR = 100
VERBOSE_EVAL = 50

def auc(y_pred, y):
    labels = y.get_label()
    y = y.get_label().astype("int")
    
    y_pred = y_pred.reshape(len(np.unique(labels)), -1)
    print(y)
    print(y_pred)
    return "auc_macro", roc_auc_score(y, y_pred, multi_class='ovr',average='macro'), True
gain = {}
nround_mean = 0
loss_list = {}
y_preds = []
for var in var_names:
    start_time = time.time()
    dtrain = lgb.Dataset(X_train[[var]], y_train.values, 
                         free_raw_data=False)
    gc.collect()
    

    for i in range(LOOP):
        gc.collect()
        extraction_cb = ModelExtractionCallback()
        callbacks = [extraction_cb,]
        
        param['seed'] = np.random.randint(9999)
        
        ret = lgb.cv(param, dtrain, NROUND,
                             nfold=NFOLD,
                             stratified=True, shuffle=True,
                             # feval=auc,
                             early_stopping_rounds=ESR, 
                             verbose_eval=VERBOSE_EVAL,
                             seed=SEED+i,callbacks=callbacks)
        bsts = extraction_cb.raw_boosters
        gain_tmp = 0
        for i in bsts:#每个i都是一个分类器
            gain_tmp += i.feature_importance('gain')#算总gain
            
        gain_tmp /= len(bsts)
        
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"Elapsed time: {elapsed_time:.2f} seconds")
        print(var)
        print(f"gain is:{gain_tmp}")
        print(f"auc is:{ret['auc_mu-mean'][-1]}")
              
        gain[var] = gain_tmp[-1]
        
        loss_list[var]= ret['auc_mu-mean'][-1]

[50]	cv_agg's auc_mu: 0.515139 + 0.0164199
[100]	cv_agg's auc_mu: 0.512634 + 0.0155662
[150]	cv_agg's auc_mu: 0.515224 + 0.0132762
[200]	cv_agg's auc_mu: 0.515698 + 0.010517
[250]	cv_agg's auc_mu: 0.515081 + 0.011085
[300]	cv_agg's auc_mu: 0.515976 + 0.0106526
Elapsed time: 2.50 seconds
sample_id
gain is:[6255.12568372]
auc is:0.5173029512654361
[50]	cv_agg's auc_mu: 0.549794 + 0.00982886
[100]	cv_agg's auc_mu: 0.545162 + 0.00494398
Elapsed time: 0.96 seconds
feature0
gain is:[5121.32922106]
auc is:0.5555575128301742
[50]	cv_agg's auc_mu: 0.51377 + 0.0132539
[100]	cv_agg's auc_mu: 0.514491 + 0.0144019
[150]	cv_agg's auc_mu: 0.515385 + 0.0125363
Elapsed time: 1.39 seconds
feature1
gain is:[4555.82228738]
auc is:0.515890980521244
[50]	cv_agg's auc_mu: 0.509788 + 0.00754924
[100]	cv_agg's auc_mu: 0.513122 + 0.00158948
Elapsed time: 0.67 seconds
feature2
gain is:[1017.46907994]
auc is:0.5153692009339496
[50]	cv_agg's auc_mu: 0.622329 + 0.00806132
[100]	cv_agg's auc_mu: 0.61965 + 0.0108266


In [11]:
imp = pd.DataFrame(list(gain.items()), columns=['feature', 'gain'])
loss_list = pd.DataFrame(list(loss_list.items()), columns=['feature', 'auc'])

imp.sort_values('gain', ascending=False, inplace=True)
imp.reset_index(drop=True, inplace=True)

loss_list.sort_values('auc', ascending=False, inplace=True)
loss_list.reset_index(drop=True, inplace=True)

In [14]:
loss_list[loss_list['feature']=='feature1']

,feature,auc
76,feature1,0.515891


In [ ]:
loss_list[loss_list['auc']>0.55]

In [ ]:
imp.to_csv(f'LOG/eachVar_gain.csv', index=False)
loss_list.to_csv(f'LOG/eachVar_auc.csv', index=False)
utils.savefig_imp(imp, f'LOG/eachVar_gain.png', x='gain',n=50)
utils.savefig_imp(loss_list, f'LOG/eachVar_auc.png', x='auc',n=50)